In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
#from nltk.corpus import stopwords
import nltk
import re
from tabulate import tabulate
from tqdm import trange

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

/home/antonio/.pyenv/versions/3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-10 15:43:28.224204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 15:43:28.224258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
import platform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os = platform.system()

match os.lower():
    case "darwin":
        path = "/Users/antoniobaio/Desktop/Progetti/ProgettiDS//BERT/IMDB.csv"
    case "linux":
        path = "/home/antonio/Scrivania/ProgettiDS/BERT/IMDB.csv"
    case "windows":
        path = "AGGIUNGI PATH"
        
print("Actual path: " + path + " and device = " + str(device))

Actual path: /home/antonio/Scrivania/ProgettiDS/BERT/IMDB.csv and device = cpu


In [12]:
df = pd.read_csv(path)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [13]:
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('english')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antonio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
df['review'] = df['review'].apply(lambda x: clean_text(x))
df['review'] = df['review'].str.replace(r'\bbr\b', '')

df

/tmp/ipykernel_5907/1984782482.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review'] = df['review'].str.replace(r'\bbr\b', '')


,review,sentiment
0,one reviewers mentioned watching oz episode ho...,positive
1,wonderful little production filming techniqu...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive
...,...,...
49995,thought movie right good job creative original...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,catholic taught parochial elementary schools n...,negative
49998,going disagree previous comment side maltin on...,negative


In [15]:
def cat2num(value):
    if value=='positive': 
        return 1
    else: 
        return 0
    
df['sentiment'] = df['sentiment'].apply(cat2num)

df = df.rename(columns={'sentiment': 'label'})

df = df.sample(frac=1, random_state=42)

df

,review,label
33553,"really liked summerslam due look arena, curtai...",1
9427,many television shows appeal quite many differ...,1
199,film quickly gets major chase scene ever incre...,0
12447,jane austen would definitely approve one gwy...,1
39489,"expectations somewhat high went see movie, tho...",0
...,...,...
11284,shadow magic recaptures joy amazement first mo...,1
44732,found movie quite enjoyable fairly entertainin...,1
38158,avoid one terrible movie exciting pointless mu...,0
860,production quite surprise absolutely love obsc...,1


In [17]:
reviews = df.review.values
labels = df.label.values

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [24]:
index=0
print(' Original: ', reviews[index])

table = np.array([tokenizer.tokenize(reviews[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews[index]))]).T

print(tabulate(table,headers = ['Tokens', 'Token IDs'],tablefmt = 'fancy_grid'))



 Original:  really liked summerslam due look arena, curtains look overall interesting reason anyways, could one best summerslam ever wwf lex luger main event yokozuna, time ok huge fat man vs strong man glad times changed terrible main event like every match luger terrible matches card razor ramon vs ted dibiase, steiner brothers vs heavenly bodies, shawn michaels vs curt hening, event shawn named big monster body guard diesel, irs vs kid, bret hart first takes doink takes jerry lawler stuff harts lawler always interesting, ludvig borga destroyed marty jannetty, undertaker took giant gonzalez another terrible match, smoking gunns tatanka took bam bam bigelow headshrinkers, yokozuna defended world title lex luger match boring terrible ending however deserves
╒═════════════╤═════════════╕
│ Tokens      │   Token IDs │
╞═════════════╪═════════════╡
│ really      │        2428 │
├─────────────┼─────────────┤
│ liked       │        4669 │
├─────────────┼─────────────┤
│ summers     │       

In [44]:
max_len = 0
l=[]
for rev in list(reviews):

    #Tokenizza il testo e aggiunge i tokens `[CLS]` e `[SEP]`
    input_ids = tokenizer.encode(rev, add_special_tokens=True)

    #Aggiorna la lunghezza massima delle frasi presenti nel dataset
    l.append(len(input_ids))
    max_len = max(max_len, len(input_ids))

if max_len>512:
  max_len=512
avg_len=int(sum(l)/len(l))

print('Max sentence length: ', max_len)
print('Avg sentence length: ', avg_len)

Max sentence length:  512
Avg sentence length:  148


In [45]:
input_ids = []
attention_masks = []

# For every tweet...
for rev in reviews:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        rev,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)



/tmp/ipykernel_5907/3879750887.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [46]:
#Stampa la prima frase come una lista di IDs
i=random.randint(0,len(reviews)-1)
print('Original: ', list(reviews)[i])
print('Input IDs:', input_ids[i])

Original:  hitting international acclaim silence lambs, director jonathan demme cut teeth making quirky comedies one like quite oscar winning american comedies could mention, fine concept, well paced, great performances, complicated romance simply funny pfeiffer mob widow moves city backwaters husband baldwin murdered crime boss killed stockwell takes fancy pfeiffer, wife reuhl furious complicate matters pfeiffer also falls cop trailing laugh minute pfeiffer, sporting hefty wig excellent widow, hyperactive ruehl modine good nice cop script simply devoid one liners, wit, humour punch lines verbal physical kind kind film demands result raises smiles best rather guffaws oozes charm, tediously short humour
Input IDs: tensor([  101,  7294,  2248, 10761,  4223, 12559,  2015,  1010,  2472,  5655,
        17183,  4168,  3013,  4091,  2437, 21864, 15952, 22092,  2028,  2066,
         3243,  7436,  3045,  2137, 22092,  2071,  5254,  1010,  2986,  4145,
         1010,  2092, 13823,  1010,  2307, 

In [47]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(reviews) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(input_ids[index]))
  token_ids = [i.numpy() for i in input_ids[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(reviews[index])
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

right, then, absolutely brilliant must intelligent quick understand humor covers attacks sorts topics, first moon landing, easter christmas, transvestitism, movies, herr doktor heimlich   averse swearing, us punctuate commas periods, uses f word also, laugh yourself, never watch feel fool   incidentally, watched stuff even saw perform live, far best work simply shines   might go far say glorious
╒═════════════╤═════════════╤══════════════════╕
│ Tokens      │   Token IDs │   Attention Mask │
╞═════════════╪═════════════╪══════════════════╡
│ [CLS]       │         101 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ right       │        2157 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ ,           │        1010 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ then        │        2059 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ ,           │        1010 │                1 │
├────────────

In [48]:
#Creaun unico Dataset (di tensori) unendo i tensori relativi agli input, alle label e alle attention mask
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

#Divide randomicamente il dataset in training set e validation set
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

40,000 training samples
10,000 validation samples


In [49]:
#Per eseguire il fine-tuning di BERT su un task specifico, è raccomandata una dimensione del batch pari a 16 o a 32
batch_size = 16

#Crea il DataLoaders per il training set e per il validation set
#Vengono presi gli elementoi del training set in ordine randomico
train_dataloader = DataLoader(
            train_dataset,  #Il training set
            sampler = RandomSampler(train_dataset), #Seleziona randomicamente i batch
            batch_size = batch_size #Esegue l'allenamento con questa dimensione dei batch
        )


validation_dataloader = DataLoader(
            val_dataset, #Il validation set
            sampler = SequentialSampler(val_dataset), #Seleziona i batch in maniera sequenziale
            batch_size = batch_size #Esegue una valutazione del modello con questa dimensione dei batch
        )

In [50]:
# Viene caricato il modello pre-allenato BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", #Usa il modello di BERT a 12-layer con un vocabolario tutto in lower case
    num_labels = 2, #Il numero delle label 
    output_attentions = False,
    output_hidden_states = False,
)

model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [51]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # learning_rate - quello standard è 5e-5
                  eps = 1e-8 # adam_epsilon  - quello standard è 1e-8.
                )

/home/antonio/.pyenv/versions/3.10.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [52]:
epochs = 2

#Numero totale degli step di training, è dato da: [numero di batch] x [numero di epoche]
total_steps = len(train_dataloader) * epochs

#Viene creato lo scheduler del learning rate
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [53]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [54]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [55]:
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

: 

: 